# [Polynomial Expansion for Orientation and Motion Estimation](http://www.diva-portal.org/smash/get/diva2:302485/FULLTEXT01.pdf)

Gunnar Farnebäck

## 4. Polynomial Expansion

Based on example signal and applicability from chapter 3.4

In [1]:
import numpy as np

**Marked point**

In [2]:
point = (2, 4)

**Signal**

In [3]:
#                       point x
#                         v
signal = np.array([[3, 7, 4, 5, 8],
                   [9, 2, 4, 4, 6],
                   [5, 1, 4, 3, 7],
                   [3, 1, 1, 2, 8],
                   [4, 6, 2, 3, 6], # < point y
                   [7, 3, 2, 6, 3],
                   [9, 6, 4, 9, 9]])

### Correlation

**Applicability**

In [4]:
applicability = np.array([[1, 2, 1],
                          [2, 4, 2],
                          [1, 2, 1]])

The applicability fixes the size of the neighborhood

In [5]:
size = applicability.shape[0]
half_size = int(size / 2)

**Polynomial basis**

{1, x, y, x², y², xy}

In [6]:
x, y = np.mgrid[-half_size : half_size + 1, -half_size : half_size + 1].reshape(2, -1)
B = np.stack([np.ones(x.size, dtype=int), x, y, x ** 2, y ** 2, x * y], axis=1)
print(B)

[[ 1 -1 -1  1  1  1]
 [ 1 -1  0  1  0  0]
 [ 1 -1  1  1  1 -1]
 [ 1  0 -1  0  1  0]
 [ 1  0  0  0  0  0]
 [ 1  0  1  0  1  0]
 [ 1  1 -1  1  1 -1]
 [ 1  1  0  1  0  0]
 [ 1  1  1  1  1  1]]


**Applicability as a n x 1 column vector**

In [7]:
a = applicability.T.reshape(-1)
print(a)

[1 2 1 2 4 2 1 2 1]


**Signal neighborhood**

In [8]:
f = signal[point[1] - half_size : point[1] + half_size + 1,
           point[0] - half_size : point[0] + half_size + 1].T.reshape(-1)
print(f)

[1 6 3 1 2 2 2 3 6]


**Resulting coefficients**

In [9]:
G = np.dot(a * B.T, B)
print(G)

[[16  0  0  8  8  0]
 [ 0  8  0  0  0  0]
 [ 0  0  8  0  0  0]
 [ 8  0  0  8  4  0]
 [ 8  0  0  4  8  0]
 [ 0  0  0  0  0  4]]


In [10]:
BWaf = np.dot(a * B.T, f)
print(BWaf)

[44 -2  8 30 18  2]


In [11]:
r = np.linalg.inv(G).dot(BWaf)
print(r)

[ 2.25 -0.25  1.    2.   -1.    0.5 ]


**Reconstructed signal projection**

In [12]:
Br = B.dot(r).reshape(size, size).T
print(Br)

[[3.   0.25 1.5 ]
 [4.5  2.25 4.  ]
 [4.   2.25 4.5 ]]


### Separable correlation

**Applicability separability**

In [13]:
a_1D = np.array([1, 2, 1])
print(np.outer(a_1D, a_1D))

[[1 2 1]
 [2 4 2]
 [1 2 1]]


**Basis separability**

In [14]:
ones = np.ones(size, dtype=int)
x = np.arange(-half_size, half_size + 1)
x2 = x ** 2
y = x
y2 = x2

B_x = np.stack([ones, x,    ones, x2,   ones, x])
B_y = np.stack([ones, ones, y,    ones, y2,   y])

BWa_x = a_1D * B_x
BWa_y = a_1D * B_y

print(np.einsum('ki,kj->ijk', B_x, B_y).reshape(size ** 2, B_x.shape[0]))

[[ 1 -1 -1  1  1  1]
 [ 1 -1  0  1  0  0]
 [ 1 -1  1  1  1 -1]
 [ 1  0 -1  0  1  0]
 [ 1  0  0  0  0  0]
 [ 1  0  1  0  1  0]
 [ 1  1 -1  1  1 -1]
 [ 1  1  0  1  0  0]
 [ 1  1  1  1  1  1]]


**Signal neighborhood**

In [15]:
f = signal[point[1] - half_size : point[1] + half_size + 1,
           point[0] - half_size : point[0] + half_size + 1]
print(f)

[[1 1 2]
 [6 2 3]
 [3 2 6]]


**Separable correlation**

In [16]:
BWaf_x = np.einsum('jk,ik', BWa_x, f)
print(BWaf_x)

[[ 5  1  5  3  5  1]
 [13 -3 13  9 13 -3]
 [13  3 13  9 13  3]]


In [17]:
BWaf = np.einsum('ij,ji->i', BWa_y, BWaf_x)
print(BWaf)

[44 -2  8 30 18  2]


**Resulting coefficients**

In [18]:
r = np.linalg.inv(G).dot(BWaf)
print(r)

[ 2.25 -0.25  1.    2.   -1.    0.5 ]


**Reconstructed signal projection**

In [19]:
Br = B.dot(r).reshape(size, size).T
print(Br)

[[3.   0.25 1.5 ]
 [4.5  2.25 4.  ]
 [4.   2.25 4.5 ]]
